# Homework 2A: Food Safety

## Cleaning and Exploring Data with Pandas



## This Assignment

In this homework, we will investigate restaurant food safety scores for restaurants in San Francisco. The scores and violation information have been [made available by the San Francisco Department of Public Health](https://data.sfgov.org/Health-and-Social-Services/Restaurant-Scores-LIVES-Standard/pyih-qa8i). The main goal for this assignment is to walk through the process of Data Cleaning and familiarise yourself with some of the `pandas` functions discussed in Pandas I and II. 


After this homework, you should be comfortable with:
* Reading CSV files 
* Reading `pandas` documentation and using `pandas`
* Working with data at different levels of granularity
* Identifying the type of data collected, missing values, anomalies, etc., and doing some basic analysis


In [5]:
import plotly
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.style.use('fivethirtyeight')

from IPython.display import display, Image 
def display_figure_for_grader(fig):
    plotly.io.write_image(fig, 'temp.png')
    display(Image('temp.png'))    

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# 0: Obtaining the Data

## File Systems and I/O


In general, we will focus on using Python commands to investigate files.  However, it can sometimes be easier to use shell commands in your local operating system.  The following cells demonstrate how to do this.

In [6]:
from pathlib import Path
data_dir = Path('.')
data_dir.mkdir(exist_ok = True)
file_path = data_dir / Path('data.zip')
dest_path = file_path

After running the cell above, if you list the contents of the directory containing this notebook, you should see `data.zip.gz`.

*Note*: The command below starts with an `!`. This tells our Jupyter Notebook to pass this command to the operating system. In this case, the command is the `ls` Unix command which lists files in the current directory.

In [7]:
!dir

 O volume na unidade C ‚ OS
 O N£mero de S‚rie do Volume ‚ ECAF-0A95

 Pasta de C:\Users\quiri\OneDrive - FEI\Trabalhos\Facul\9ø semestre\Tratamento de Dados\Tratamento de Dados\Projeto I

29/03/2025  16:31    <DIR>          .
27/03/2025  22:17    <DIR>          ..
27/03/2025  19:37    <DIR>          data
27/03/2025  17:26           609.879 data.zip
29/03/2025  16:31            82.393 hw02A (1).ipynb
               2 arquivo(s)        692.272 bytes
               3 pasta(s)   11.687.796.736 bytes dispon¡veis


## Loading Food Safety Data

We have data, but we don't have any specific questions about the data yet. Let's focus on understanding the structure of the data; this involves answering questions such as:

* Is the data in a standard format or encoding?
* Is the data organized in records?
* What are the fields in each record? (We sometimes also use the term 'feature' or 'attribute' as well, depending on the context)

Let's start by looking at the contents of `data.zip`. It's not just a single file but rather a compressed directory of multiple files. We could inspect it by uncompressing it using a shell command such as `!unzip data.zip`, but in this homework, we're going to do almost everything in Python for maximum portability.

## Looking Inside and Extracting the Zip Files

The following code blocks are for setup. Simply run the cells; **do not modify them**. Question 1a is where you will start to write code.

Here, we assign `my_zip` to a `zipfile.Zipfile` object representing `data.zip`, and assign `list_names` to a list of all the names of the contents in `data.zip`.

In [8]:
import zipfile
my_zip = zipfile.ZipFile(dest_path, 'r')
list_names = my_zip.namelist()
list_names

['data/',
 'data/bus.csv',
 'data/ins.csv',
 'data/ins2vio.csv',
 'data/vio.csv',
 'data/sf_zipcodes.json',
 'data/legend.csv']

You may notice that we did not write `zipfile.ZipFile('data.zip', ...)`. Instead, we used `zipfile.ZipFile(dest_path, ...)`. In general, we **strongly suggest having your filenames hard coded as string literals only once** in a notebook. It is very dangerous to hardcode things twice because if you change one but forget to change the other, you can end up with bugs that are very hard to find.

Now, we display the files' names and their sizes.

In [9]:
my_zip = zipfile.ZipFile(dest_path, 'r')
for info in my_zip.infolist():
    print('{}\t{}'.format(info.filename, info.file_size))

data/	0
data/bus.csv	665365
data/ins.csv	1860919
data/ins2vio.csv	1032799
data/vio.csv	4213
data/sf_zipcodes.json	474
data/legend.csv	120


Often when working with zipped data, we'll never unzip the actual zip file. This saves space on our local computer. However, for this homework the files are small, so we're just going to unzip everything. This has the added benefit that you can look inside the CSV files using a text editor, which might be handy for understanding the structure of the files. The cell below will unzip the CSV files into a sub-directory called `data`.

In [10]:
data_dir = Path('.')
my_zip.extractall(data_dir)
!dir {data_dir / Path("data")}

 O volume na unidade C ‚ OS
 O N£mero de S‚rie do Volume ‚ ECAF-0A95

 Pasta de C:\Users\quiri\OneDrive - FEI\Trabalhos\Facul\9ø semestre\Tratamento de Dados\Tratamento de Dados\Projeto I\data

27/03/2025  19:37    <DIR>          .
29/03/2025  16:31    <DIR>          ..
29/03/2025  16:31           665.365 bus.csv
29/03/2025  16:31         1.860.919 ins.csv
29/03/2025  16:31         1.032.799 ins2vio.csv
29/03/2025  16:31               120 legend.csv
29/03/2025  16:31               474 sf_zipcodes.json
29/03/2025  16:31             4.213 vio.csv
               6 arquivo(s)      3.563.890 bytes
               2 pasta(s)   11.688.951.808 bytes dispon¡veis


The cell above created a folder called `data`, and in it there should be five CSV files. Let's open up `legend.csv.gz` to see its contents. To do this, click on the file icon on the top left to show the folders and files within the hw02A folder, then click on `legend.csv.gz`. The file will open up in another tab. You should see something that looks like:

    "Minimum_Score","Maximum_Score","Description"
    0,70,"Poor"
    71,85,"Needs Improvement"
    86,90,"Adequate"
    91,100,"Good"

The `legend.csv.gz` file does indeed look like a well-formed CSV file. Let's check the other three files. Rather than opening up each file manually, let's use Python to print out the first 5 lines of each. We defined a helper function for you that will allow you to retrieve the first N lines of a file as a list. For example, `head('data/legend.csv.gz', 5)` will return the first 5 lines of "data/legend.csv.gz". Run the cell below to print out the first 5 lines of all six files that we just extracted from the zip file.

In [13]:
import os

def head(filename, lines=5):
    '''
    Returns the first few lines of a file.
     
    filename: the name of the file to open
    lines: the number of lines to include
    
    return: A list of the first few lines from the file.
    '''

    from itertools import islice
    with open(filename, "r") as f:
        return list(islice(f, lines))

data_dir = "./"
for f in list_names:
    if not os.path.isdir(f):
        print(head(data_dir + f, 5), "\n")

['"business id column","name","address","city","state","postal_code","latitude","longitude","phone_number"\n', '"1000","HEUNG YUEN RESTAURANT","3279 22nd St","San Francisco","CA","94110","37.755282","-122.420493","-9999"\n', '"100010","ILLY CAFFE SF_PIER 39","PIER 39  K-106-B","San Francisco","CA","94133","-9999","-9999","+14154827284"\n', '"100017","AMICI\'S EAST COAST PIZZERIA","475 06th St","San Francisco","CA","94103","-9999","-9999","+14155279839"\n', '"100026","LOCAL CATERING","1566 CARROLL AVE","San Francisco","CA","94124","-9999","-9999","+14155860315"\n'] 

['"iid","date","score","type"\n', '"100010_20190329","03/29/2019 12:00:00 AM","-1","New Construction"\n', '"100010_20190403","04/03/2019 12:00:00 AM","100","Routine - Unscheduled"\n', '"100017_20190417","04/17/2019 12:00:00 AM","-1","New Ownership"\n', '"100017_20190816","08/16/2019 12:00:00 AM","91","Routine - Unscheduled"\n'] 

['"iid","vid"\n', '"97975_20190725","103124"\n', '"85986_20161011","103114"\n', '"95754_2019032

## Reading in and Verifying Data

Based on the above information, let's attempt to load `bus.csv`, `ins2vio.csv`, `ins.csv`, and `vio.csv` into `pandas` `DataFrame`s with the following names: `bus`, `ins2vio`, `ins`, and `vio`, respectively.

*Note:* Because of character encoding issues, one of the files (`bus`) will require an additional argument `encoding='ISO-8859-1'` when calling `pd.read_csv`. At some point in your future, you should read all about [character encodings](https://diveintopython3.problemsolving.io/strings.html). We won't discuss these in detail in Data 100.

In [19]:
# Path to the directory containing data
dsDir = Path('data')

bus = pd.read_csv(dsDir/'bus.csv', encoding='ISO-8859-1')
ins2vio = pd.read_csv(dsDir/'ins2vio.csv')
ins = pd.read_csv(dsDir/'ins.csv')
vio = pd.read_csv(dsDir/'vio.csv')

# This code is essential for the autograder to function properly. Do not edit
ins_test = ins

Now that you've read the files, let's try some `pd.DataFrame` methods ([docs](https://pandas.pydata.org/pandas-docs/version/1.4.3/reference/api/pandas.DataFrame.html)).
Use the `DataFrame.head` method to show the top few lines of the `bus`, `ins`, and `vio` `DataFrame`s. For example, running the cell below will display the first few lines of the `bus` `DataFrame`. 

In [16]:
bus.head()

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675


To show multiple return outputs in one single cell, you can use `display()`. 

In [17]:
display(bus.head())
display(ins.head())

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675


,iid,date,score,type
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup


The `DataFrame.describe` method can also be handy for computing summaries of numeric columns of our `DataFrame`s. Try it out with each of our 4 `DataFrame`s. Below, we have used the method to give a summary of the `bus` `DataFrame`. 

In [20]:
bus.describe()

,business id column,latitude,longitude,phone_number
count,6253.000000,6253.000000,6253.000000,6.253000e+03
mean,60448.948984,-5575.337966,-5645.817699,4.701819e+09
std,36480.132445,4983.390142,4903.993683,6.667508e+09
min,19.000000,-9999.000000,-9999.000000,-9.999000e+03
25%,18399.000000,-9999.000000,-9999.000000,-9.999000e+03
50%,75685.000000,-9999.000000,-9999.000000,-9.999000e+03
75%,90886.000000,37.776494,-122.421553,1.415533e+10
max,102705.000000,37.824494,0.000000,1.415988e+10


Now, we perform some sanity checks for you to verify that the data was loaded with the correct structure.

First, we check the basic structure of the `DataFrame`s you created:

In [21]:
assert all(bus.columns == ['business id column', 'name', 'address', 'city', 'state', 'postal_code',
                           'latitude', 'longitude', 'phone_number'])
assert 6250 <= len(bus) <= 6260

assert all(ins.columns == ['iid', 'date', 'score', 'type'])
assert 26660 <= len(ins) <= 26670

assert all(vio.columns == ['description', 'risk_category', 'vid'])
assert 60 <= len(vio) <= 65

assert all(ins2vio.columns == ['iid', 'vid'])
assert 40210 <= len(ins2vio) <= 40220

Next we'll check that the statistics match what we expect. The following are hard-coded statistical summaries of the correct data.

In [22]:
bus_summary = pd.DataFrame(**{'columns': ['business id column', 'latitude', 'longitude'],
 'data': {'business id column': {'50%': 75685.0, 'max': 102705.0, 'min': 19.0},
  'latitude': {'50%': -9999.0, 'max': 37.824494, 'min': -9999.0},
  'longitude': {'50%': -9999.0,
   'max': 0.0,
   'min': -9999.0}},
 'index': ['min', '50%', 'max']})

ins_summary = pd.DataFrame(**{'columns': ['score'],
 'data': {'score': {'50%': 76.0, 'max': 100.0, 'min': -1.0}},
 'index': ['min', '50%', 'max']})

vio_summary = pd.DataFrame(**{'columns': ['vid'],
 'data': {'vid': {'50%': 103135.0, 'max': 103177.0, 'min': 103102.0}},
 'index': ['min', '50%', 'max']})

from IPython.display import display

print('What we expect from your Businesses DataFrame:')
display(bus_summary)
print('What we expect from your Inspections DataFrame:')
display(ins_summary)
print('What we expect from your Violations DataFrame:')
display(vio_summary)

What we expect from your Businesses DataFrame:


,business id column,latitude,longitude
min,19.0,-9999.000000,-9999.0
50%,75685.0,-9999.000000,-9999.0
max,102705.0,37.824494,0.0


What we expect from your Inspections DataFrame:


,score
min,-1.0
50%,76.0
max,100.0


What we expect from your Violations DataFrame:


,vid
min,103102.0
50%,103135.0
max,103177.0


The code below defines a testing function that we'll use to verify that your data has the same statistics as what we expect. Run these cells to define the function. The `df_allclose` function has this name because we are verifying that all of the statistics for your `DataFrame` are close to the expected values. Why not `df_allequal`? It's a bad idea in almost all cases to compare two floating point values like 37.780435, as rounding errors can cause spurious failures. Run the following cells to load some basic utilities (you do not need to change these at all):

In [23]:
"""Run this cell to load this utility comparison function that we will use in various
tests below (both tests you can see and those we run internally for grading).

Do not modify the function in any way.
"""


def df_allclose(actual, desired, columns=None, rtol=5e-2):
    """Compare selected columns of two Dataframes on a few summary statistics.
    
    Compute the min, median and max of the two Dataframes on the given columns, and compare
    that they match numerically to the given relative tolerance.
    
    If they don't match, an AssertionError is raised (by `numpy.testing`).
    """    
    # Summary statistics to compare on
    stats = ['min', '50%', 'max']
    
    # For the desired values, we can provide a full DF with the same structure as
    # the actual data, or pre-computed summary statistics.
    # We assume a pre-computed summary was provided if column is None. In that case, 
    # `desired` *must* have the same structure as the actual's summary
    if columns is None:
        des = desired
        columns = desired.columns
    else:
        des = desired[columns].describe().loc[stats]

    # Extract summary stats from actual DF
    act = actual[columns].describe().loc[stats]

    return np.allclose(act, des, rtol)

We will now explore each file in turn, including determining its granularity and exploring many of the variables individually. Let's begin with the businesses file, which has been read into the `bus` `DataFrame`.

<br/><br/><br/>

---

<br/><br/><br/>

# Question 1: Examining the Business Data File

## Question 1a

From its name alone, we expect the `bus.csv` file to contain information about the restaurants. Let's investigate the granularity of this dataset.

In [24]:
bus.head()

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675


The `bus` `DataFrame` contains a column called `business id`, which probably corresponds to a unique business id.  However, we will first rename that column to `bid` for simplicity.

**Note**: In practice, we might want to do this renaming when the table is loaded, but for grading purposes, we will do it here.


In [25]:
bus = bus.rename(columns={"business id column": "bid"})

Examining the entries in `bus`, is the `bid` unique for each record (i.e. each row of data)? Your code should compute the answer, i.e. don't just hard code `True` or `False`.

**Hint**: Use [`value_counts()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) or [`unique()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html) to determine if the `bid` series has any duplicates.

In [31]:
is_bid_unique = bus.value_counts(bus['bid'])
is_bid_unique

bid
102705    1
19        1
24        1
31        1
45        1
         ..
101       1
99        1
98        1
95        1
88        1
Name: count, Length: 6253, dtype: int64

<br/><br/><br/>

---


## Question 1b

We will now work with some important fields in `bus`.

1. Assign `top_names` to an iterable containing the top 6 most frequently used business names, from most frequent to least frequent. 
2. Assign `top_addresses` to an iterable containing the top 6 addresses where businesses are located, from most popular to least popular.
   
Recall from CS88 or CS61A that "an iterable object is anything that can be passed to the built-in iter function. Iterables include sequence values such as strings and tuples, and other containers such as sets and dictionaries."

**Hint 1**: You may find [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) helpful.

**Hint 2**: You'll need to get the names / addresses, NOT the counts associated with each. Some way to **reset the index** would come in handy. If you're unsure how to do this, try looking through the class notes or using a search engine. Part of the goal of this course is to develop independent thinking in the context of the data science lifecycle, which can involve a fair bit of exploring and reading documentation. It may be a bit annoying at first, but you'll get the hang of it, and we're here to guide you on that path! 

**Hint 3**: To check your answer, `top_names[0]` should return the string `Peet's Coffee & Tea`. It should not be a number.


In [61]:
top_names = list(bus.value_counts('name').nlargest(n=6).index)
top_addresses = list(bus.value_counts('address').nlargest(n=6).index)

display(top_names)
display(top_addresses)

["Peet's Coffee & Tea",
 'Starbucks Coffee',
 'Jamba Juice',
 "McDonald's",
 'Proper Food',
 'STARBUCKS']

['Off The Grid',
 '428 11th St',
 '3251 20th Ave',
 '2948 Folsom St',
 'Pier 41',
 '103 Horne Ave']

name
Peet's Coffee & Tea              20
Starbucks Coffee                 13
Jamba Juice                      10
McDonald's                       10
Proper Food                       9
                                 ..
Zutuhil Azteca                    1
Zynga                             1
Zzan                              1
iNoodles                          1
Target Store T-3201-Starbucks     1
Name: count, Length: 5775, dtype: int64

<br/><br/><br/>

---


## Question 1c

Based on the above exploration, what does each record represent?

**A**. A city block.

**B**. A chain of restaurants.

**C**. One location of a restaurant.

Answer in the following cell. Your answer should be a string, either `"A"`, `"B"`, or `"C"`.


In [ ]:
# What does each record represent?  Valid answers are:
#    "A"
#    "B"
#    "C"
q1c = "A"

<br/><br/><br/>

---

<br/><br/><br/>

# 2: Cleaning the Business Data Postal Codes

The business data contains postal code information that we can use to aggregate the ratings over regions of the city. Let's examine and clean the postal code field. The postal code (sometimes also called a [ZIP code](https://en.wikipedia.org/wiki/ZIP_Code)) partitions the city into regions:

<img src="https://usmapguide.com/wp-content/uploads/printable-san-francisco-zip-code-map.jpg" alt="ZIP Code Map" style="width: 600px">

<br/><br/>

---


## Question 2a

How many restaurants are in each ZIP code? 

In the cell below, create a **Series** where the index is the postal code and the value is the number of records with that postal code. The Series should be in descending order of count. Do you notice any odd/invalid ZIP codes?


In [264]:
zip_counts = bus.value_counts('postal_code')
print(zip_counts.to_string())

postal_code
94103         562
94110         555
94102         456
94107         408
94133         398
94109         382
94111         259
94122         255
94105         249
94118         231
94115         230
94108         229
94124         218
94114         200
-9999         194
94112         192
94117         189
94123         177
94121         157
94104         142
94132         132
94116          97
94158          90
94134          82
94127          67
94131          49
94130           8
94143           5
94013           2
94101           2
94301           2
94188           2
CA              2
94120           1
941102019       1
94105-2907      1
94117-3504      1
94105-1420      1
941033148       1
94080           1
94014           1
92672           1
64110           1
00000           1
941             1
94102-5917      1
94124-1917      1
94129           1
94122-1909      1
94123-3106      1
94518           1
94602           1
94621           1
94901           1
94544           

<br/><br/><br/>

--- 

## Question 2b

In Question 2a, we noticed a large number of potentially invalid ZIP codes (e.g., "Ca"). These are likely due to data entry errors. To get a better understanding of the potential errors in the zip codes, let's break down the problem into two parts.

### Part I

Import a list of valid San Francisco ZIP codes by using `pd.read_json` to load the file `data/sf_zipcodes.json`, and store them as a Series in `valid_zips`. As you may expect, `pd.read_json` works similarly to `pd.read_csv` but for JSON files (a different file format you'll learn more about in HW 3) that you can read more about [here](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html). If you are unsure of what data type a variable is, remember you can do `type(some_var_name)` to check!

In [170]:
valid_zips = pd.read_json('data/sf_zipcodes.json')
valid_zips

,zip_codes
0,94102
1,94103
2,94104
3,94105
4,94107
5,94108
6,94109
7,94110
8,94111
9,94112


Observe that `pd.read_json` reads data as integers by default. This isn't quite what we want! We would like to store ZIP codes as strings (you'll learn more about why soon!). To do that, we can use the `astype` function to generate a copy of the `pandas` `Series` stored as strings instead.

In [185]:
valid_zips = valid_zips.astype("string")
valid_zips

,zip_codes
0,94102
1,94103
2,94104
3,94105
4,94107
5,94108
6,94109
7,94110
8,94111
9,94112


If you're ever unsure about the data type of a variable, remember you can always check using the `type` function like below:

In [186]:
type(valid_zips)

pandas.core.frame.DataFrame

Now it's time to do Part II. You will probably want to use the `Series.isin` function. For more information on this function see the [the documentation linked in this internet search](https://www.google.com/search?q=series+isin+pandas&rlz=1C1CHBF_enUS910US910&oq=series+isin+pandas&aqs=chrome..69i57l2j69i59j69i60l2j69i65j69i60l2.1252j0j7&sourceid=chrome&ie=UTF-8). 

**Note:** You are welcome and, in fact, encouraged to search and read documentation on the internet to complete the assignments in the course, even if the documentation is not linked explicitly.

### Part II

 Construct a `DataFrame` containing only the businesses which DO NOT have valid ZIP codes.

In [383]:
lista = ~bus['postal_code'].isin(valid_zips['zip_codes'])

invalid_zip_bus = bus[lista]
invalid_zip_bus.head(20)

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number
22,100126,Lamas Peruvian Food Truck,Private Location,San Francisco,CA,-9999,-9999.000000,-9999.000000,-9999
68,100417,"COMPASS ONE, LLC",1 MARKET ST. FL,San Francisco,CA,94105-1420,-9999.000000,-9999.000000,14154324000
96,100660,TEAPENTER,1518 IRVING ST,San Francisco,CA,94122-1909,-9999.000000,-9999.000000,14155868318
109,100781,LE CAFE DU SOLEIL,200 FILLMORE ST,San Francisco,CA,94117-3504,-9999.000000,-9999.000000,14155614215
144,101084,Deli North 200,1 Warriors Way Level 300 North East,San Francisco,CA,94518,-9999.000000,-9999.000000,-9999
156,101129,Vendor Room 200,1 Warriors Way Level 300 South West,San Francisco,CA,-9999,-9999.000000,-9999.000000,-9999
177,101192,Cochinita #2,2 Marina Blvd Fort Mason,San Francisco,CA,-9999,-9999.000000,-9999.000000,14150429222
276,102014,"DROPBOX (Section 3, Floor 7)",1800 Owens St,San Francisco,CA,-9999,-9999.000000,-9999.000000,-9999
295,102245,Vessell CA Operations (#4),2351 Mission St,San Francisco,CA,-9999,-9999.000000,-9999.000000,-9999
298,10227,The Napper Tandy,3200 24th St,San Francisco,CA,-9999,37.752581,-122.416482,-9999


<br/><br/><br/>

--- 

## Question 2c

In the previous question, many of the businesses had a common invalid postal code that was likely used to encode a MISSING postal code.  Do they all share a potentially "interesting address"? For that purpose, in the following cells, we will construct a series that counts the number of businesses at each `address` that have this single likely MISSING postal code value. 

Let's break this down into steps: 

### Part 1
Identify the single common missing postal code and assign it to `missing_postal_code`. Then create a `DataFrame`, `bus_missing`, to store only those businesses in `bus` that have `missing_postal_code` as their postal code.

In [401]:
missing_postal_code = invalid_zip_bus.value_counts('postal_code').index[0]
bus_missing = invalid_zip_bus[invalid_zip_bus['postal_code'] == missing_postal_code]
bus_missing

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number
22,100126,Lamas Peruvian Food Truck,Private Location,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
156,101129,Vendor Room 200,1 Warriors Way Level 300 South West,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
177,101192,Cochinita #2,2 Marina Blvd Fort Mason,San Francisco,CA,-9999,-9999.0,-9999.0,14150429222
276,102014,"DROPBOX (Section 3, Floor 7)",1800 Owens St,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
295,102245,Vessell CA Operations (#4),2351 Mission St,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
...,...,...,...,...,...,...,...,...,...
5820,96774,DANIEL SCOTT RIPLEY CATERER,425 Barneveld Ave,San Francisco,CA,-9999,-9999.0,-9999.0,14155212714
5843,96938,POKE REESE,3251 Pierce,San Francisco,CA,-9999,-9999.0,-9999.0,14150463411
5940,97700,THE ITALIAN HOMEMADE COMPANY,1 franklin Ct,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
6078,98788,333 Truck,Off The Grid,San Francisco,CA,-9999,-9999.0,-9999.0,-9999


### Part 2
Using `bus_missing`, find the number of businesses at each address (which would all share the same postal code). Specifically, `missing_zip_address_count` should store a Series with addresses as the indices and the counts as the values.

In [402]:
missing_zip_address_count = pd.Series(bus_missing.value_counts('address'))
missing_zip_address_count.head()

address
Off The Grid          39
Off the Grid          10
OTG                    4
Approved Locations     3
OFF THE GRID           3
Name: count, dtype: int64

<!-- BEGIN QUESTION -->

<br/><br/><br/>

--- 

## Question 2d

If we were to drop businesses with postal code values equal to `missing_postal_code`, what specific types of businesses would we be excluding? In other words, is there a commonality among businesses with missing postal codes?

**Hint**: You may want to identify and Google the names of the businesses with missing postal codes. Feel free to reuse parts of your code from 2c to re-examine `bus_missing`, but we will not be grading your code.


São todos negócios que fecharam, por isso não possuem mais um código zip

<!-- END QUESTION -->

<br/><br/><br/>

--- 

## Question 2e

Examine the `invalid_zip_bus` `DataFrame` we computed in Question 2c and look at the businesses that DO NOT have the special MISSING ZIP code value. Some of the invalid postal codes are just the full 9-digit code rather than the first 5 digits. Create a new column named `postal5` in the original `bus` `DataFrame` which contains only the first 5 digits of the `postal_code` column.

Then, for any of the `postal5` ZIP code entries that were not a valid San Francisco ZIP code (according to `valid_zips`), the provided code will set the `postal5` value to `None`. 

**Hint:** You will find `str` accessors particularly useful. They allow you to use your usual Python string functions in tandem with a `DataFrame`. For example, if you wanted to use the `replace` function on every entry in a column of a `DataFrame` to change the letter 'a' to 'e', you could do so by writing `df['col_name'].str.replace('a', 'e')`. Think about the different ways you can extract the first 5 digits using regular Python code!

**Do not modify the provided code! Simply add your own code in place of the ellipses.**


In [554]:
bus['postal5'] = None

bus['postal5'] = bus['postal_code'].str[0:5]

bus.loc[~bus['postal5'].isin(valid_zips['zip_codes']), 'postal5'] = None
# Checking the corrected postal5 column
bus.loc[invalid_zip_bus.index, ['bid', 'name', 'postal_code', 'postal5']]

,bid,name,postal_code,postal5
22,100126,Lamas Peruvian Food Truck,-9999,None
68,100417,"COMPASS ONE, LLC",94105-1420,94105
96,100660,TEAPENTER,94122-1909,94122
109,100781,LE CAFE DU SOLEIL,94117-3504,94117
144,101084,Deli North 200,94518,None
...,...,...,...,...
6173,99369,HOTEL BIRON,94102-5917,94102
6174,99376,Mashallah Halal Food truck Ind,-9999,None
6199,99536,FAITH SANDWICH #2,94105-2907,94105
6204,99681,Twister,95112,None


<br/><br/><br/>

---

<br/><br/><br/>

# 3: Investigate the Inspection Data

Let's now turn to the inspection `DataFrame`. Earlier, we found that `ins` has 4 columns named 
`iid`, `score`, `date`, and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

Let's start by looking again at the first 5 rows of `ins` to see what we're working with.

In [555]:
ins.head(5)

,iid,date,score,type,bid,timestamp,year
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010,2019-03-29,2019
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,100017,2019-04-17,2019
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,100017,2019-08-26,2019


<br/><br/><br/>

---

## Question 3a

The column `iid` probably corresponds to an inspection id.  Write an expression (line of code) that evaluates to `True` or `False` based on whether all the inspection ids are unique.

**Hint:** This is a very similar question to Question 1a.

In [556]:
is_ins_iid_unique = ins.value_counts(ins['iid']) == 1
is_ins_iid_unique 

iid
99_20180808        True
100010_20190329    True
100010_20190403    True
100017_20190417    True
100017_20190816    True
                   ... 
100036_20190906    True
100036_20190531    True
100036_20190325    True
100030_20190826    True
100030_20190612    True
Name: count, Length: 26663, dtype: bool

<br/><br/>

---

## Question 3b

We would like to extract `bid` from each row of the `ins` `DataFrame`. If we look carefully, the column `iid` of the `ins` `DataFrame` appears to be the composition of two numbers and the first number looks like a business id.  

Create a new column called `bid` in the `ins` Dataframe containing just the business id.  You will want to use `ins['iid'].str` operations to do this. (Python's in-built `split` method could come in use, read up on the documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html)!) Also, be sure to convert the type of this column to `int`. 

**Hint**: Similar to an earlier problem where we used `astype("string")` to convert a column to a string, here you should use `astype` to convert the `bid` column into type `int`. **No Python `for` loops or list comprehensions are allowed.** This is on the honor system since our autograder isn't smart enough to check, but if you're using `for` loops or list comprehensions, you're doing the HW incorrectly. 

In [557]:
ins['bid'] = ins['iid'].str.partition('_')[0].astype("int")
ins.head(5)

,iid,date,score,type,bid,timestamp,year
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010,2019-03-29,2019
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,100017,2019-04-17,2019
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,100017,2019-08-26,2019


<br/><br/><br/>

---

## Question 3c

For this part, we're going to explore some new somewhat strange syntax that we haven't seen in lecture. Don't panic! If you're not sure what to do, try experimenting, Googling, and don't shy away from talking to other students or course staff.

For this problem we'll use the time component of the inspection data.  All of this information is given in the `date` column of the `ins` `DataFrame`. 

**No Python `for` loops or list comprehensions are allowed!**

### Part I

What is the type of the individual `ins['date']` entries? You may want to grab the very first entry and use the `type` function in Python. 

In [558]:
ins_date_type = type(ins['date'])
ins_date_type

pandas.core.series.Series

### Part II
Rather than the type you discovered in Part 1, we want each entry in `pd.TimeStamp` format. You might expect that the usual way to convert something from it current type to `TimeStamp` would be to use `astype`. You can do that, but the more typical way is to use `pd.to_datetime`. Using `pd.to_datetime`, create a new `ins['timestamp']` column containing `pd.Timestamp` objects.  These will allow us to do date manipulation with much greater ease in part III and part IV. 

Note: You may run into a UserWarning in case you do not specify the date format when using `pd.to_datetime`. To resolve this, consider using the following string '%m/%d/%Y %I:%M:%S %p' to specify the `format`.

In [559]:
ins['timestamp'] = pd.to_datetime(ins['date'], format= '%m/%d/%Y %I:%M:%S %p', dayfirst=True)
ins['timestamp'] 

0       2019-03-29
1       2019-04-03
2       2019-04-17
3       2019-08-16
4       2019-08-26
           ...    
26658   2018-09-24
26659   2018-11-02
26660   2019-09-09
26661   2017-12-07
26662   2018-08-08
Name: timestamp, Length: 26663, dtype: datetime64[ns]

### Part III

What are the earliest and latest dates in our inspection data?  

**Hint**: you can use `min` and `max` on dates of the correct type.

In [560]:
earliest_date = ins['timestamp'].min(0)
latest_date = ins['timestamp'].max(0)
print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)

Earliest Date: 2016-10-04 00:00:00
Latest Date: 2019-11-28 00:00:00


### Part IV

We probably want to examine the inspections by year. Create an additional `ins['year']` column containing just the year of the inspection.  Consider using `pd.Series.dt.year` to do this.

In case you're curious, the documentation for `TimeStamp` data can be found at [this link](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp).

In [561]:
ins['year'] = ins['timestamp'].dt.year
ins['year']

0        2019
1        2019
2        2019
3        2019
4        2019
         ... 
26658    2018
26659    2018
26660    2019
26661    2017
26662    2018
Name: year, Length: 26663, dtype: int32

<br/><br/><br/>

---

<br/><br/><br/>

# 4: Some Analysis

Let's try and figure out whether there are any differences between names of restaurants located in even and odd ZIP codes (specifically using the 5-digit postal codes). We will break down this analysis into steps with the end goal of figuring out the restaurant with the longest name and a valid phone number, among all the even ZIP codes and odd ZIP codes respectively.

## Question 4a

First, create a new column `name_length` that stores the length of the `name` of each of the restaurants in `bus`. Again, **do not use for loops or list comprehensions**.

In [650]:
bus['name_length'] = bus['name'].str.replace(" ", "").str.len()
bus[['name', 'name_length']]

,name,name_length
0,HEUNG YUEN RESTAURANT,19
1,ILLY CAFFE SF_PIER 39,18
2,AMICI'S EAST COAST PIZZERIA,24
3,LOCAL CATERING,13
4,OUI OUI! MACARON,14
...,...,...
6248,SUSIECAKES BAKERY,16
6249,HINODEYA SOMA,12
6250,TON TON,6
6251,URBAN EXPRESS KITCHENS LLC,23


<br/><br/>

---

## Question 4b

To work the 5-digit ZIP codes and check whether they are even or odd, we need to ensure that there are no None values contained. Create a new `DataFrame` `bus_valid` which only contains rows with `postal5` values that are not None. You may find the `.isna()` function useful! For the rest of this question, we will be working with `bus_valid`.

In [538]:
help(pd.isna)

Help on function isna in module pandas.core.dtypes.missing:

isna(obj: 'object') -> 'bool | npt.NDArray[np.bool_] | NDFrame'
    Detect missing values for an array-like object.

    This function takes a scalar or array-like object and indicates
    whether values are missing (``NaN`` in numeric arrays, ``None`` or ``NaN``
    in object arrays, ``NaT`` in datetimelike).

    Parameters
    ----------
    obj : scalar or array-like
        Object to check for null or missing values.

    Returns
    -------
    bool or array-like of bool
        For scalar input, returns a scalar boolean.
        For array input, returns an array of boolean indicating whether each
        corresponding element is missing.

    See Also
    --------
    notna : Boolean inverse of pandas.isna.
    Series.isna : Detect missing values in a Series.
    DataFrame.isna : Detect missing values in a DataFrame.
    Index.isna : Detect missing values in an Index.

    Examples
    --------
    Scalar arguments (in

In [651]:
bus_valid = bus[bus['postal5'].notnull()]
bus_valid.head()

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999,94110,19
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284,94133,18
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839,94103,24
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315,94124,13
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675,94124,14


<br/><br/>

---

## Question 4c

Now, assign `is_even` to a boolean **Series** that indicates whether the corresponding 5-digit ZIP code in `bus_valid` is even or odd. Remember to keep in mind the data type of `postal5`!

Hint: You might find the mod operation `%` useful here!

In [652]:
is_even = bus_valid['postal5'].astype("int") % 2 == 0
is_even

0        True
1       False
2       False
3        True
4        True
        ...  
6248     True
6249    False
6250     True
6251    False
6252     True
Name: postal5, Length: 6032, dtype: bool

<br/><br/>

---

## Question 4d 

Using the Series you created above, store the name of the business with the shortest name amongst all businesses located in even ZIP codes in `shortest_name_even`. You do not have to use the skeleton code provided and can use more/fewer lines than provided, but make sure that `shortest_name_even` contains a **string** with your answer. 

Note: When sorting, please break ties alphabetically. Feel free to reference the [sort_values documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) to see how you can sort by multiple values. 

In [571]:
# bus_valid["even"] = (bus_valid["postal5"].astype(int).map(lambda x: x%2) ==0)
# sorted = bus_valid[(bus_valid["even"] == True)].sort_values("name_length", ascending=True)
# shortest_name_even = sorted.head(1)["name"].values[0]
# shortest_name_even

In [692]:
shortest_name_even = bus_valid[is_even].loc[bus_valid[is_even]['name'].str.len().idxmin(), 'name']
print(f'Menor nome: {shortest_name_even}')

bus_valid[is_even].sort_values(['name_length', "name"], ascending = True)

Menor nome: B44


,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length
5810,96670,903,1566 Carroll Ave,San Francisco,CA,94124,-9999.000000,-9999.000000,14155671581,94124,3
589,1831,B44,44 Belden Pl,San Francisco,CA,94104,37.791278,-122.403748,-9999,94104,3
6211,99717,Dum,3111 24th St,San Francisco,CA,94110,-9999.000000,-9999.000000,-9999,94110,3
3065,71962,ENO,320 Geary St,San Francisco,CA,94102,-9999.000000,-9999.000000,14159306257,94102,3
5982,97938,GAI,3463 16th St,San Francisco,CA,94114,-9999.000000,-9999.000000,14151232538,94114,3
...,...,...,...,...,...,...,...,...,...,...,...
626,18952,Hilton Financial District- Restaurant Seven Fifty,750 Kearny St 1st Floor,San Francisco,CA,94108,37.795057,-122.404876,-9999,94108,44
628,18954,Hilton Financial District-Flute Coffee and Win...,750 Kearny St 1st Floor,San Francisco,CA,94108,37.795057,-122.404876,-9999,94108,45
4652,88829,Namaskar Foods Authentic Nepalese and Indian C...,3251 20th Ave,San Francisco,CA,94132,-9999.000000,-9999.000000,14155703214,94132,45
1931,5812,Zaida T. Rodriguez (ZTR) Annex Child Developme...,421 BARTLETT St,San Francisco,CA,94110,37.750243,-122.419293,14155695844,94110,48


<br/><br/>

---

## Question 4e 

Suppose we only consider businesses in `bus_valid` that also have valid phone numbers along with being located in even ZIP codes. That is, we no longer include businesses that have invalid phone numbers. Does your answer from 4d change? Here, an invalid phone number refers to the single common missing value for phone numbers, similar to the `missing_postal_code` you found in 2b. 

Write an expression that indicates whether the shortest name is the same (True) or not (False) if we require that the business must have a valid phone number. Feel free to use the scratch cell below if need be! You may find your code from the previous part to be a useful starting point. Again, please make sure to break ties alphabetically.

In [704]:
# SCRATCH CELL

missing_postal_code = invalid_zip_bus.value_counts('postal_code').index[0]
bus_missing = invalid_zip_bus[invalid_zip_bus['postal_code'] == missing_postal_code]

num_errado = bus_valid.value_counts('phone_number').index[0]
bus_missing_num = bus_valid[bus_valid['phone_number'] == num_errado]

shortest_name_tel = bus_missing_num.loc[bus_missing_num['name'].str.len().idxmin(), 'name']
print(f'Menor nome: {shortest_name_tel}')

bus_missing_num.sort_values(['name_length', "name"], ascending = True)

Menor nome: ZA


,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length
2228,61861,SO,1010 BRYANT St,San Francisco,CA,94103,37.772264,-122.407339,-9999,94103,2
705,2053,ZA,1919 Hyde St,San Francisco,CA,94109,37.798553,-122.418964,-9999,94109,2
5571,95022,ALX,680 Folsom St,San Francisco,CA,94107,-9999.000000,-9999.000000,-9999,94107,3
589,1831,B44,44 Belden Pl,San Francisco,CA,94104,37.791278,-122.403748,-9999,94104,3
1277,3520,Bix,56 Gold St,San Francisco,CA,94111,37.796778,-122.402875,-9999,94111,3
...,...,...,...,...,...,...,...,...,...,...,...
1409,37168,InterContinental San Francisco 3rd Fl. Banquet...,888 Howard St 3rd Floor,San Francisco,CA,94103,37.781664,-122.404778,-9999,94103,48
5520,94868,California Pacific Medical Center-Mission Bern...,3555 Cesar Chavez St,San Francisco,CA,94110,-9999.000000,-9999.000000,-9999,94110,50
1886,5578,Simco/Wipe Out Bar & Grill(Hot Dog Cart)- inf...,Pier 39 A02,San Francisco,CA,94133,-9999.000000,-9999.000000,-9999,94133,51
5354,93888,505 Pinterest Dining Garage Level Kitchen and ...,505 Brannan St Garage Level,San Francisco,CA,94105,-9999.000000,-9999.000000,-9999,94105,54


In [706]:
same_shortest_name = shortest_name_tel == shortest_name_even
same_shortest_name

False

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

In [707]:
bus = pd.read_csv('data/bus.csv', encoding='ISO-8859-1').rename(columns={"business id column": "bid"})
bus['postal5'] = bus['postal_code'].str[:5]
ins = pd.read_csv('data/ins.csv')
ins['timestamp'] = pd.to_datetime(ins['date'], format='%m/%d/%Y %I:%M:%S %p')
ins['bid'] = ins['iid'].str.split("_", expand=True)[0].astype(int) 

# This code is essential for the autograder to function properly. Do not edit.
ins_test = ins

<br/><br/>

---


# Question 5: Inspecting the Inspections


## Question 5a

Let's start by looking again at the first 5 rows of `ins` to see what we're working with.

In [735]:
ins.head(5)

,iid,date,score,type,timestamp,bid
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,2019-03-29,100010
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,2019-04-17,100017
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,2019-08-26,100017


To better understand how the scores have been allocated, examine how the maximum score varies for each type of inspection. Create a `DataFrame` object `ins_score_by_type`, indexed by all the inspection types (e.g., New Construction, Routine - Unscheduled, etc.), with a single column named `max_score` containing the highest score received. You may find `df.rename()` to be useful!

In [776]:
ins_score_by_type = ins.groupby('type')['score'].max()
ins_score_by_type.rename('max_score')

type
Administrative or Document Review     -1
Community Health Assessment           -1
Complaint                             -1
Complaint Reinspection/Followup       -1
Foodborne Illness Investigation       -1
Multi-agency Investigation            -1
New Construction                      -1
New Ownership                         -1
New Ownership - Followup              -1
Non-inspection site visit             -1
Reinspection/Followup                 -1
Routine - Scheduled                   -1
Routine - Unscheduled                100
Special Event                         -1
Structural Inspection                 -1
Name: max_score, dtype: int64

<br/>

---

## Question 5b


Given the variability of `ins['score']` observed in 1a, let's examine the inspection scores `ins['score']` further.

In [772]:
ins['score'].value_counts().head()

score
-1      12632
 100     1993
 96      1681
 92      1260
 94      1250
Name: count, dtype: int64

There are a large number of inspections with a `score` of `-1`. These are probably missing values. Let's see what types of inspections have scores and which do not (score of -1).  We have defined for you a new column `'Missing Score'` that shows `True` if the score for that business is `-1` to help you out with the analysis. 

Use `.groupby` to find out the number of scores that every combination of `type` and `Missing Score` can take on. The result should be a **`DataFrame`** that should look **exactly** as shown below:

<center> <img src="pics/1b.png" width="400"/> 


In [782]:
ins['Missing Score'] = (ins['score'] == -1).astype("str")
ins_missing_score_group = ins.groupby('type')['Missing Score'].size()
ins_missing_score_group

type
Administrative or Document Review        4
Community Health Assessment              1
Complaint                             1458
Complaint Reinspection/Followup        227
Foodborne Illness Investigation        115
Multi-agency Investigation               3
New Construction                       994
New Ownership                         1592
New Ownership - Followup               499
Non-inspection site visit              811
Reinspection/Followup                 6439
Routine - Scheduled                     46
Routine - Unscheduled                14077
Special Event                            3
Structural Inspection                  394
Name: Missing Score, dtype: int64

<br/>

---

## Question 5c


Using `.groupby` to perform the above analysis gave us a `DataFrame` that wasn't the most readable at first glance. There are better ways to represent the above information that take advantage of the fact that we are looking at combinations of two variables. It's time to pivot (pun intended)!

Create the following `DataFrame`, and assign it to to the variable `ins_missing_score_pivot`. You'll want to use the `pivot_table` method of the `DataFrame` class, which you can read about in the `pivot_table` [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html). Once you create `ins_missing_score_pivot`, add another column titled `'Total'`, which contains the total number of inspections of that `type`. Sort the table by descending order of `'Total'`.

**Hint:** Consider what happens if there are no values that correspond to a particular combination of `'Missing Score'` and `'type'`. Looking at the documentation for `pivot_table`, is there any function argument that allows you to specify what value to fill in?

If you've done everything right, you should observe that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections and that `ins_missing_score_pivot` looks exactly like below:


<table border="1" class="dataframe" >  <thead>    
    <tr style="text-align: right;">      <th>Missing Score</th>      <th>False</th>      <th>True</th>      <th>Total</th>    </tr>    <tr align="right">      <th>type</th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    
    <tr  align="right">      <th>Routine - Unscheduled</th>      <td>14031</td>      <td>46</td>      <td>14077</td>    </tr>    
    <tr  align="right">      <th>Reinspection/Followup</th>      <td>0</td>      <td>6439</td>      <td>6439</td>    </tr>    
    <tr  align="right">      <th>New Ownership</th>      <td>0</td>      <td>1592</td>      <td>1592</td>    </tr>    
    <tr  align="right">      <th>Complaint</th>      <td>0</td>      <td>1458</td>      <td>1458</td>    </tr>    
    <tr  align="right">      <th>New Construction</th>      <td>0</td>      <td>994</td>      <td>994</td>    </tr>    
    <tr  align="right">      <th>Non-inspection site visit</th>      <td>0</td>      <td>811</td>      <td>811</td>    </tr>    
    <tr  align="right">      <th>New Ownership - Followup</th>      <td>0</td>      <td>499</td>      <td>499</td>    </tr>    
    <tr  align="right">      <th>Structural Inspection</th>      <td>0</td>      <td>394</td>      <td>394</td>    </tr>    
    <tr  align="right">      <th>Complaint Reinspection/Followup</th>      <td>0</td>      <td>227</td>      <td>227</td>    </tr>    
    <tr  align="right">      <th>Foodborne Illness Investigation</th>      <td>0</td>      <td>115</td>      <td>115</td>    </tr>    
    <tr  align="right">      <th>Routine - Scheduled</th>      <td>0</td>      <td>46</td>      <td>46</td>    </tr>    
    <tr  align="right">      <th>Administrative or Document Review</th>      <td>0</td>      <td>4</td>      <td>4</td>    </tr>    
    <tr  align="right">      <th>Multi-agency Investigation</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    
    <tr  align="right">      <th>Special Event</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    
    <tr  align="right">      <th>Community Health Assessment</th>      <td>0</td>      <td>1</td>      <td>1</td>    </tr>  </tbody></table>


In [802]:
ins['Missing Score'] = (ins['score'] == -1).astype("str")
ins_missing_score_pivot = pd.pivot_table(ins[['type', 'Missing Score']], index='type', columns='Missing Score', values='Missing Score', aggfunc=len, fill_value=0)

ins_missing_score_pivot['Total'] = ins_missing_score_pivot['False'] + ins_missing_score_pivot['True']
ins_missing_score_pivot = ins_missing_score_pivot.sort_values('Total', ascending=False)

ins_missing_score_pivot 

Missing Score,False,True,Total
type,,,
Routine - Unscheduled,14031,46,14077
Reinspection/Followup,0,6439,6439
New Ownership,0,1592,1592
Complaint,0,1458,1458
New Construction,0,994,994
Non-inspection site visit,0,811,811
New Ownership - Followup,0,499,499
Structural Inspection,0,394,394
Complaint Reinspection/Followup,0,227,227


Notice that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections. It is reasonable for inspection types such as `New Ownership` and `Complaint` to have no associated inspection scores, but we might be curious why there are no inspection scores for the `Reinspection/Followup` inspection type.

<br/><br/>

---

# Question 6: Joining Data Across Tables

In this question, we will start to connect data across multiple tables. We will be using the `merge` function. 

<br/>

--- 

## Question 6a

Let's figure out which restaurants had the lowest scores. Before we proceed, let's filter out missing scores from `ins` so that negative scores don't influence our results. 

Note that there might be something interesting we could learn from businesses with missing scores, but we are omitting such analysis from this homework. You might consider exploring this for the optional question at the end. 

Note: We have no idea if there is actually anything interesting to learn as we have not attempted this ourselves.

In [804]:
ins = ins[ins["score"] > 0]
ins

,iid,date,score,type,timestamp,bid,Missing Score
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010,False
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017,False
15,100041_20190520,05/20/2019 12:00:00 AM,83,Routine - Unscheduled,2019-05-20,100041,False
20,100055_20190425,04/25/2019 12:00:00 AM,98,Routine - Unscheduled,2019-04-25,100055,False
21,100055_20190912,09/12/2019 12:00:00 AM,82,Routine - Unscheduled,2019-09-12,100055,False
...,...,...,...,...,...,...,...
26654,999_20170714,07/14/2017 12:00:00 AM,77,Routine - Unscheduled,2017-07-14,999,False
26656,999_20180123,01/23/2018 12:00:00 AM,80,Routine - Unscheduled,2018-01-23,999,False
26660,999_20190909,09/09/2019 12:00:00 AM,80,Routine - Unscheduled,2019-09-09,999,False
26661,99_20171207,12/07/2017 12:00:00 AM,82,Routine - Unscheduled,2017-12-07,99,False


We'll start by creating a new `DataFrame` called `ins_named`. It should be exactly the same as `ins`, except that it should have the name and address of every business, as determined by the `bus` `DataFrame`. 

**Hint**: Use the `merge` method to join the `ins` `DataFrame` with the appropriate portion of the `bus` `DataFrame`. See the official [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) on how to use `merge`. The first few rows of the resulting `DataFrame` you create are shown below:

<img src="pics/2a.png" width="1080"/>

In [814]:
ins_named = ins.merge(bus).loc[:, :'address'] 
ins_named.head()

,iid,date,score,type,timestamp,bid,Missing Score,name,address
0,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010,False,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B
1,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017,False,AMICI'S EAST COAST PIZZERIA,475 06th St
2,100041_20190520,05/20/2019 12:00:00 AM,83,Routine - Unscheduled,2019-05-20,100041,False,UNCLE LEE CAFE,3608 BALBOA ST
3,100055_20190425,04/25/2019 12:00:00 AM,98,Routine - Unscheduled,2019-04-25,100055,False,Twirl and Dip,335 Martin Luther King Jr. Dr
4,100055_20190912,09/12/2019 12:00:00 AM,82,Routine - Unscheduled,2019-09-12,100055,False,Twirl and Dip,335 Martin Luther King Jr. Dr


<br/>

--- 

## Question 6b

Let's look at the 20 businesses in `ins_named` with the lowest **median** score. Order your results by the median score followed by the business name to break ties. The resulting table should look like the table below.

This one is pretty challenging! Don't forget to rename the `score` column. 

**Hint**: The `agg` function can accept a dictionary as an input. See the [agg documentation](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html). Additionally, when thinking about what aggregation functions to use, ask yourself what value would be in the `"name"` column for each entry across the group? Can we select just one of these values to represent the whole group?

As usual, **YOU SHOULD NOT USE LOOPS OR LIST COMPREHENSIONS**. Try and break down the problem piece by piece instead, gradually chaining together different `pandas` functions. Feel free to use more than one line!

<table border="1" class="dataframe">  <thead>    
    <tr style="text-align: right;">      <th></th>      <th>name</th>      <th>median score</th>    </tr> 
    <tr  align="right">  <th align="right">bid</th>      <th></th>      <th></th>    </tr> </thead>  <tbody>    
    <tr  align="right">      <th>84590</th>      <td>Chaat Corner</td>      <td>54.0</td>    </tr>    
    <tr  align="right">        <th>90622</th>      <td>Taqueria Lolita</td>      <td>57.0</td>    </tr>    
    <tr  align="right">         <th>94351</th>      <td>VBowls LLC</td>      <td>58.0</td>    </tr>    
    <tr  align="right">          <th>69282</th>      <td>New Jumbo Seafood Restaurant</td>      <td>60.5</td>    </tr>    
    <tr  align="right">         <th>1154</th>      <td>SUNFLOWER RESTAURANT</td>      <td>63.5</td>    </tr>  
    <tr  align="right">          <th>93150</th>      <td>Chez Beesen</td>      <td>64.0</td>    </tr>   
    <tr  align="right">     <th>39776</th>      <td>Duc Loi Supermarket</td>      <td>64.0</td>    </tr>  
    <tr  align="right">         <th>78328</th>      <td>Golden Wok</td>      <td>64.0</td>    </tr>  
    <tr  align="right">          <th>69397</th>      <td>Minna SF Group LLC</td>      <td>64.0</td>    </tr>     
    <tr  align="right">        <th>93502</th>      <td>Smoky Man</td>      <td>64.0</td>    </tr>    
    <tr  align="right">           <th>98995</th>      <td>Vallarta's Taco Bar</td>      <td>64.0</td>    </tr>    
    <tr  align="right">         <th>10877</th>      <td>CHINA FIRST INC.</td>      <td>64.5</td>    </tr>    
    <tr  align="right">        <th>71310</th>      <td>Golden King Vietnamese Restaurant</td>      <td>64.5</td>    </tr>     
    <tr  align="right">          <th>89070</th>      <td>Lafayette Coffee Shop</td>      <td>64.5</td>    </tr>
    <tr  align="right">          <th>71008</th>      <td>House of Pancakes</td>      <td>65.0</td>    </tr> 
    <tr  align="right">         <th>2542</th>      <td>PETER D'S RESTAURANT</td>      <td>65.0</td>    </tr>           
    <tr  align="right">        <th>3862</th>      <td>IMPERIAL GARDEN SEAFOOD RESTAURANT</td>      <td>66.0</td>    </tr>   
    <tr  align="right">         <th>61427</th>      <td>Nick's Foods</td>      <td>66.0</td>    </tr>    
    <tr  align="right">          <th>72176</th>      <td>Wolfes Lunch</td>      <td>66.0</td>    </tr>    
    <tr  align="right">        <th>89141</th>      <td>Cha Cha Cha on Mission</td>      <td>66.5</td>    </tr>  </tbody></table>

In [920]:
twenty_lowest_scoring = ins_named.groupby('name').agg({'score': 'median'}).sort_values('score').head(21)
twenty_lowest_scoring = twenty_lowest_scoring.rename(columns={'score': 'median score'})
twenty_lowest_scoring

,median score
name,
Chaat Corner,54.0
Taqueria Lolita,57.0
VBowls LLC,58.0
New Jumbo Seafood Restaurant,60.5
SUNFLOWER RESTAURANT,63.5
Duc Loi Supermarket,64.0
Smoky Man,64.0
Golden Wok,64.0
Chez Beesen,64.0


<br/>

--- 
## Question 6c

Let's figure out which restaurant had the worst score ever (single lowest score). 

In the cell below, assign `worst_restaurant` to the name of the restaurant with the **lowest inspection score ever**. We should not be considering restaurants with missing scores, so this should not be a retaurant that has a score of `-1`. For fun: Look up the reviews for this restaurant on Yelp. Do you see any reviews that indicate this restaurant had health inspection issues?


In [907]:
worst_restaurant = ins_named.groupby('name', as_index=False).min().sort_values('score').iloc[0]['name']
worst_restaurant

'Lollipot'

<br/>

--- 
## Question 6d

Did this restaurant clean up its act? Look in the database to see if it scored better on its next inspection. Assign `cleaned_up` to `True` or `False`, depending on whether it performed better or not.

In [918]:
# SCRATCH WORK
ins_named[ins_named['name'] == 'Lollipot']

,iid,date,score,type,timestamp,bid,Missing Score,name,address
10897,86718_20161116,11/16/2016 12:00:00 AM,90,Routine - Unscheduled,2016-11-16,86718,False,Lollipot,890 Taraval St
10898,86718_20180522,05/22/2018 12:00:00 AM,45,Routine - Unscheduled,2018-05-22,86718,False,Lollipot,890 Taraval St
10899,86718_20181005,10/05/2018 12:00:00 AM,90,Routine - Unscheduled,2018-10-05,86718,False,Lollipot,890 Taraval St


In [ ]:
cleaned_up = True

<br/><br/>

---
# Question 7: Let Them Eat Cake! 

Now that you've analyzed and found out which restaurants to avoid in SF, we can turn toward the more interesting question of what dessert places are the best! For the purposes of this question, we assume that cake is the best dessert (and rightfully so!)

<br/>

--- 
## Question 7a
 
In your quest to find the best cake shop, the first step is to find all the businesses in `ins_named` that **contain the word 'cake'** in their `name`, and assign the resulting `DataFrame` to `cake_shops`. To help you out, we created the `lowercase_name` column so you do not need to worry about checking for capitalized letters when checking if `name` contains `'cake'`.

**Hint:** You might find the `.str` accessors useful yet again!

In [927]:
ins_named['lowercase_name'] = ins_named['name'].str.lower()
cake_shops = ins_named[ins_named['lowercase_name'].str.contains('cake')]
cake_shops

,iid,date,score,type,timestamp,bid,Missing Score,name,address,lowercase_name
1637,26340_20170424,04/24/2017 12:00:00 AM,75,Routine - Unscheduled,2017-04-24,26340,False,VIP Coffee & Cake Shop,671 Broadway St,vip coffee & cake shop
1638,26340_20180514,05/14/2018 12:00:00 AM,85,Routine - Unscheduled,2018-05-14,26340,False,VIP Coffee & Cake Shop,671 Broadway St,vip coffee & cake shop
1911,2898_20161103,11/03/2016 12:00:00 AM,94,Routine - Unscheduled,2016-11-03,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
1912,2898_20180412,04/12/2018 12:00:00 AM,94,Routine - Unscheduled,2018-04-12,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
1913,2898_20190816,08/16/2019 12:00:00 AM,96,Routine - Unscheduled,2019-08-16,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
1945,29304_20180112,01/12/2018 12:00:00 AM,92,Routine - Unscheduled,2018-01-12,29304,False,Kara's Cupcakes,3249 SCOTT,kara's cupcakes
2172,3210_20170110,01/10/2017 12:00:00 AM,96,Routine - Unscheduled,2017-01-10,3210,False,International House of Pancakes,200 Beach St,international house of pancakes
2173,3210_20170809,08/09/2017 12:00:00 AM,98,Routine - Unscheduled,2017-08-09,3210,False,International House of Pancakes,200 Beach St,international house of pancakes
2174,3210_20180205,02/05/2018 12:00:00 AM,96,Routine - Unscheduled,2018-02-05,3210,False,International House of Pancakes,200 Beach St,international house of pancakes
2175,3210_20181106,11/06/2018 12:00:00 AM,98,Routine - Unscheduled,2018-11-06,3210,False,International House of Pancakes,200 Beach St,international house of pancakes


<br/>

--- 
## Question 7b

Assign `cake_at_least_3` to a `DataFrame` consisting of only those cake shops that have had at least 3 inspections. Remember, the `bid` uniquely defines a cake shop, not its `name`!

In [956]:
cake_at_least_3 = cake_shops[cake_shops['bid'].isin((cake_shops['bid'].value_counts() >= 3).index)]
cake_at_least_3.head()

,iid,date,score,type,timestamp,bid,Missing Score,name,address,lowercase_name
1637,26340_20170424,04/24/2017 12:00:00 AM,75,Routine - Unscheduled,2017-04-24,26340,False,VIP Coffee & Cake Shop,671 Broadway St,vip coffee & cake shop
1638,26340_20180514,05/14/2018 12:00:00 AM,85,Routine - Unscheduled,2018-05-14,26340,False,VIP Coffee & Cake Shop,671 Broadway St,vip coffee & cake shop
1911,2898_20161103,11/03/2016 12:00:00 AM,94,Routine - Unscheduled,2016-11-03,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
1912,2898_20180412,04/12/2018 12:00:00 AM,94,Routine - Unscheduled,2018-04-12,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
1913,2898_20190816,08/16/2019 12:00:00 AM,96,Routine - Unscheduled,2019-08-16,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory


<!-- BEGIN QUESTION -->

<br/>

---
## Question 7c

In the cell below, run the following line of code: `q3c_df = cake_at_least_3.sort_values('timestamp').groupby('bid').agg('first')`

Is the granularity of `cake_at_least_3` the same as the granularity of `q3c_df`? In other words, what does a single row of `q3c_df` represent, and what does a single row in `cake_at_least_3` represent? Explain the granularity of each `DataFrame`. Your answer does not need to be more than 2-3 lines, but you should be specific. 

**Note**: For more details on what the granularity of a `DataFrame` means, feel free to check [Section 5.2.1](https://ds100.org/course-notes/eda/eda.html#granularity) in the course notes! 

In [957]:
q3c_df = cake_at_least_3.sort_values('timestamp').groupby('bid').agg('first')
q3c_df.head()

,iid,date,score,type,timestamp,Missing Score,name,address,lowercase_name
bid,,,,,,,,,
386,386_20171116,11/16/2017 12:00:00 AM,84,Routine - Unscheduled,2017-11-16,False,ADELITA'S CAKES,3780 MISSION St,adelita's cakes
2898,2898_20161103,11/03/2016 12:00:00 AM,94,Routine - Unscheduled,2016-11-03,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
3210,3210_20170110,01/10/2017 12:00:00 AM,96,Routine - Unscheduled,2017-01-10,False,International House of Pancakes,200 Beach St,international house of pancakes
4630,4630_20170304,03/04/2017 12:00:00 AM,96,Routine - Unscheduled,2017-03-04,False,Zanze's Cheesecake,2405 Ocean Ave,zanze's cheesecake
7757,7757_20170131,01/31/2017 12:00:00 AM,92,Routine - Unscheduled,2017-01-31,False,CAKE GALLERY,290 09th St,cake gallery


Ambos os Dataframes possuem o mesmo nível de granularidade, cada linha dos dois contém as informação de uma loja de bolos. O maior diferencial é na maneira em que os dados estão organizados, o Dataframe q3c_df está organizado para mostrar as primeiras inspeções primeiros 

<!-- END QUESTION -->

<br/>

---
## Question 7d

Rather than the inspection scores, you find that the number of vowels present in the business `name` is a better indicator of how good the cake is when it comes to the shops in `cake_at_least_3`. Using the helper function `count_vowels` we have defined for you, sort all the cake shops in `cake_at_least_3` based on the number of vowels in the business's name in descending order. Then, return a **Python `list`** consisting of the top 2 **uniquely named** cake shops using this sorted `DataFrame`. You should break ties using alphabetical ordering. You do not need to stick to the skeleton code provided, but you are **not allowed to do not add any new columns!**

This is pretty challenging, but rest assured, the price of knowing the best cake shops is well worth it! 

**Hint**: When working on this problem, it might be helpful to check out [Section 4.1](https://ds100.org/course-notes/pandas_3/pandas_3.html) in the course notes which touches on custom sorts! 

In [958]:
def count_vowels(name):
    vowels = 'aeiouAEIOU'
    count = 0
    return sum([letter in vowels for letter in name])

In [972]:
sorted_by_vowel_count = cake_at_least_3['name'].map(count_vowels)
top_2_cake = sorted_by_vowel_count

top_2_cake
#sorted_by_vowel_count

1637      7
1638      7
1911      8
1912      8
1913      8
1945      5
2172     13
2173     13
2174     13
2175     13
2328      5
2329      5
2330      5
3128      3
3129      3
3268      6
3269      6
3674      7
3675      7
3676      7
6179      6
6180      6
6262      5
6263      5
7098      7
7099      7
7100      7
7372      7
7373      7
7374      7
7475      6
7476      6
7477      6
8308      4
8309      4
8507      4
8508      4
8509      4
8510      4
8518      6
8519      6
8520      6
8546      5
8547      5
8548      5
10513     5
10514     5
11313     8
11314     8
11315     8
12451     9
12452     9
13527     7
13635     6
13990     6
Name: name, dtype: int64

<!-- BEGIN QUESTION -->

<br/>

---
## Question 7e

Finally, to examine different parts of a chained `pandas` statement, describe the purpose of each of the functions used (`.loc`, `.groupby`, `idxmax()`) in words. 

Secondly, share what you think this line of code accomplishes. In other words, write a question that could be answered using this statement.

While the first part of this question will be graded for correctness, the second part of this question is a bit more open-ended. Answers demonstrating your understanding will get full credit.

In [967]:
cake_at_least_3.loc[cake_at_least_3.groupby("bid")["score"].idxmax()].head()

,iid,date,score,type,timestamp,bid,Missing Score,name,address,lowercase_name
3268,386_20171116,11/16/2017 12:00:00 AM,84,Routine - Unscheduled,2017-11-16,386,False,ADELITA'S CAKES,3780 MISSION St,adelita's cakes
1913,2898_20190816,08/16/2019 12:00:00 AM,96,Routine - Unscheduled,2019-08-16,2898,False,The Cheesecake Factory,"251 Geary St, 8th Floor",the cheesecake factory
2173,3210_20170809,08/09/2017 12:00:00 AM,98,Routine - Unscheduled,2017-08-09,3210,False,International House of Pancakes,200 Beach St,international house of pancakes
3675,4630_20180629,06/29/2018 12:00:00 AM,100,Routine - Unscheduled,2018-06-29,4630,False,Zanze's Cheesecake,2405 Ocean Ave,zanze's cheesecake
8510,7757_20190829,08/29/2019 12:00:00 AM,94,Routine - Unscheduled,2019-08-29,7757,False,CAKE GALLERY,290 09th St,cake gallery


.loc é uma função usada pegar um grupo de linhas e colonuas em um Dataset.

.groupby é uma função usada para organizar os dados de um Dataset, com base nos parâmetros que inserir, em grupos.

.idxmax é uma função usada para retornar quando o valor máximo aconteceu pela primeira vez.

Crie um Dataset que contenha apenas as pontuações máximas de score para cada loja com base em seu código bid.

In [ ]:
# You may do some scratch work in this cell, however, only your written answer will be graded. 
# Any outputs or dataframes you generate here will not be counted as part of your explanation.

<!-- END QUESTION -->

<br/><br/>

---

# Question 8: Restaurant Ratings Over Time

As a final challenge, we consider a scenario involving restaurants with multiple ratings over time.

Let's see which restaurant location has had the most extreme improvement in its scores. Let the "swing" of a restaurant location be defined as the difference between its highest-ever and lowest-ever score. **Only consider restaurant locations with at least 3 scores—that is, restaurants that were rated at least 3 times.** Assign `max_swing` to the name of the restaurant that has the maximum swing. 

We have not provided any skeleton, as there are many paths to getting the correct answer. The recommended approach to solving this problem is to break it down into smaller chunks (e.g., first, ensure all restaurants have at least 3 scores; second, compute the swing, etc.). This will likely require more than one line, so feel free to add/remove columns and define new temporary variables. Remember to assign your solution - a string containing the `name` of the restaurant location that experienced the most extreme improvement - to `max_swing` after you do so. 

**Note**: The "swing" is of a specific restaurant location. There might be some restaurants with multiple locations; we are focusing on the swing of a particular restaurant as specified by its `name` and `address`.

In [1013]:
# Pegando os restaurantes com 3 ou mais notas
restaurantes3scores = ins_named[ins_named['bid'].isin((cake_shops['bid'].value_counts() >= 3).index)]
# Separo somente as três colunas importantes
restaurantes3scores = restaurantes3scores[['name', 'address', 'score']]

# Então faço um Dataset organizado de acordo com a média das notas, e assumo que o primeiro seja o max_swing
max_swing = restaurantes3scores.groupby(['name', 'address']).agg({'score': 'mean'}).sort_values('score').head(1)

max_swing

,,score
name,address,
House of Pancakes,937 TARAVAL,69.0


## Summary of Inspections Data

We have done a lot in this homework! 
 
- Broke down the inspection scores in detail using `.groupby` and `pivot_table`
- Joined the business and inspection data and identified the name of the restaurant with the worst rating
- Took a deep dive into the sweet world of cake and found the best spots under varying metrics
- Took a swing at analyzing how restaurant inspection scores change over time!

Over the course of this 2-part homework, we hope you have become more familiar with `pandas` - in terms of identifying when to use particular functions, how they work, when they can support EDA - as well as with EDA and Data Cleaning, as part of the broader Data Science Lifecycle. These tools will serve you well as a data scientist!